# All
The argument to the ALL function must be either a reference to a base table or a reference to a base column. You cannot use table expressions or column expressions with the ALL function.

Return value
The table or column with filters removed.

## All(Table)

```
Removes all filters from the specified table. In effect, ALL(Table) returns all of the values in the table, removing any filters from the context that otherwise might have been applied. This function is useful when you are working with many levels of grouping, and want to create a calculation that creates a ratio of an aggregated value to the total value. The first example demonstrates this scenario.
```

### Example:
![Alt text](all_filter.png)

### Card with total value
Total Sales = SUMX(ALL(Sales),Sales[Sales Amount]) 

### Table 1 with percentage from total
>% Sales = SUMX(Sales,Sales[Sales Amount])/SUMX(ALL(Sales),Sales[Sales Amount])

### Table 2 with percentage contribution of each product for each year (no filter on year while there is filtering by business type). This can be achieved by one of the two: ALL('Date'[Fiscal Year]) OR ALLEXCEPT('Date'[business Type]).
>% Sales_year = SUMX(Sales,Sales[Sales Amount])/CALCULATE(SUM(Sales[Sales Amount]),ALL('Date'[Fiscal Year]))

### Table 3 with percentage contribution of each year for each product sales (no filter on business type while there is filtering by year). This can be achieved by one of the two: ALL('Date'[Business Type]) OR ALLEXCEPT('Date'[Fiscal Year]).
% Sales_type = SUMX(Sales,Sales[Sales Amount])/CALCULATE(SUM(Sales[Sales Amount]),ALL(Reseller[Business Type]))

# Summary of filtering by Calculate and SUMX

|Tab1\Tab2 |Tab2 Val 1|Tab2 Val 2|Subtotal|
|----------|----------|----------|--------|
|Tab1 Val 1|          |          |        |
|Tab1 Val 2|          |          |        |
|Subtotal  |          |          |Total   |

## Calculate/SUMX:
Note Tab=Table; Fact table is where the calculated value is located
1. ALL(Fact): no filter on all columns meaning return total.
2. ALL(Tab1): breakdown all values by column from Tab2. 
3. ALL(Tab2): breakdown all values by column from Tab1. 
4. ALL(Tab1 Column): breakdown all values by column from Tab2.
5. ALLEXCEPT(Tab1, Column): apply all filters by both columns.
6. ALL(Tab3): apply all filters by both columns.

See example below

![Alt text](All%20functions%20Summary.png)

# ALLNOBLANKROW
From the parent table of a relationship, returns all rows but the blank row, or all distinct values of a column but the blank row, and disregards any context filters that might exist.

# ALLSELECTED
When ALL has been applied to remove filters in the above tables, value for total remains the same when we apply external filter. Sometimes, we need to calculate total for selected filters. In that case, ALLSELECTED is used to indicate total should be calculated for values related to external filters only. In the example below, country is the external filter and 'selected sales' calculates total for applied external filter:
>Selected Sales = CALCULATE(sum(Sales[Sales Amount]),ALLSELECTED('Sales Territory'[Country]))
![Alt text](allselected.png)

# Filter, FILTER(VALUES), KEEPFILTERS

# __Note that it is generally note recommended to use filter in tables since it will slow down performance especially in large tables since filter function takes entire tables and runs it to get to result which adds work load.__ 
>Instead use keepfilters. Note that for multiple values, use values along with filter. Otherwise, all three give the same result.

# KEEPFILTERS (the prefered option of the three)
Modifies how filters are applied while evaluating a CALCULATE or CALCULATETABLE function.
You use KEEPFILTERS within the context CALCULATE and CALCULATETABLE functions, to override the standard behavior of those functions. Meaning the total for the applied filter will be shown for the filters columns as specified by within calculate function. See exampke below:

Sales_filter = CALCULATE(SUM(Sales[Sales Amount]),'Sales Territory'[Country]="Canada",'Product'[Category]="Bikes")

Sales_filter_keepfilters = CALCULATE(SUM(Sales[Sales Amount]),KEEPFILTERS('Sales Territory'[Country]="Canada"),KEEPFILTERS('Product'[Category]="Bikes"))

Sales_table_filter = CALCULATE(SUM(Sales[Sales Amount]),FILTER('Sales Territory','Sales Territory'[Country]="Canada"),FILTER('Product','Product'[Category]="Bikes"))

Sales_table_filter_value = CALCULATE(SUM(Sales[Sales Amount]),FILTER(VALUES('Sales Territory'),'Sales Territory'[Country]="Canada"),FILTER(VALUES('Product'),'Product'[Category]="Bikes"))

![Alt text](keepfilters,%20filter,%20filter+value.png)

# Removefilters
Remove filter removes all filters as specified by 
1. a table: REMOVEILFTER(Product)
2. a column: REMOVEILFTER(Product[Category]),
3. any filter: REMOVEFILTER()

>Note that REMOVEFILTERS is equivalent to ALL but much more efficient since it doesn't need to refer to a table in the background. 

Top table indicates REMOVEFILTER removing filters from product category: \
__Sales_removefilters_product = CALCULATE(SUM(Sales[Sales Amount]),REMOVEFILTERS('Product'))__

Middle table indicates REMOVEFILTER removing filters from product category which is not located in products table but related to sales table through product table.

Bottom table indicates REMOVEFILTER not filtering sales values by country since the removefilter instuction is specified for product table only and not sales terittory table. But sales calculated using the removeall filters indicates that nothing filters the value: neither product nor country \
__sales_remove all filters = CALCULATE(sum(Sales[Sales Amount]),REMOVEFILTERS())__

![Alt text](removefilters.png)

# SELECTEDVALUE and VALUES
SELECTEDVALUE is used to select one value while VALUES is for multiple values.
The purpose of these functions is to capture value(s) selected by enduser and use it for conditional formating without filtering a visual.

To do that, duplicate a filter of interest in a separate table that is not joined to any table in the model. That way, enduser will be able to click on slicer from the duplicate value and a visual will be formated accordingly.

Here are codes for sales total:
>1. SELECTEDVALUE 


Total Sales_Selectedvalue = 


VAR selected=SELECTEDVALUE('Table 2'[Category slicer])


return


IF(SELECTEDVALUE('Product'[Category])=selected,SUM(Sales[Sales Amount]))


>2. VALUES returns a table


Total Sales_values = 


VAR selected=VALUES('Table 2'[Category slicer])


return


IF(SELECTEDVALUE('Product'[Category]) in selected,SUM(Sales[Sales Amount]))

1 is used to do conditional formating using single value while 2 is used to do formating on 1 or more values. See figures below:
![Alt text](selectedvalue.png)
![Alt text](values.png)

# ALLCROSSFILTERED

Clear all filters which are applied to a table.


total_sales = CALCULATE(SUM(Sales[Sales Amount]), ALLCROSSFILTERED(Sales))

![Alt text](allcrossfiltered.png)

# FILTERS

> Filter takes only one argument, which is the name of a column.

> It returns all values that are actively being filtered. Calculations involving a column from a related table will work.


Example 1: Sales by Product Colour

Sales by Colour Filters = 

SUMMARIZE(
    
        FILTERS(DimProduct[ColorName]),

        DimProduct[ColorName],
        
        "Sales Amount", FactOnlineSales[Sales_Amount]
        
        )

> Filtering a FILTERS argument using a cross filter column will not work

Example 2: Colour Filters has the same result as Colour Filters_corssfiltered by Product Category

Colour Filters = 

SUMMARIZE(
    
    FILTERS(DimProduct[ColorName]),
    
    DimProduct[ColorName],"Count",COUNTROWS(DimProduct)
    
    )


Colour Filters_cross_filtered = 

SUMMARIZE(
    
    CALCULATETABLE(
        
        FILTERS(
            
            DimProduct[ColorName]),
            
            DimProductCategory[ProductCategoryName]="Audio"),
            
            DimProduct[ColorName],"Count",COUNTROWS(DimProduct)
            
        )

|ColorName      |	Count|
|---------------|--------|
|Silver         |	417|
|Blue           |	200|
|White          |	505|
|Red            |	99|
|Black          |	602|
|Green          |	74|
|Orange         |	55|
|Pink           |	84|
|Yellow         |	36|
|Purple         |	6|
|Brown          |	77|
|Grey           |	283|
|Gold           |	50|
|Azure          |	14|
|Silver Grey    |	14|
|Transparent    |	1|

> However, value in FILTERS can be filtered by values on the column within FILTERS

Example 3:

Colour Filters_within_filtered = 

SUMMARIZE(
    
    CALCULATETABLE(
        
        FILTERS(
            
            DimProduct[ColorName]),
            
            DimProduct[ColorName]="Azure"),
            
            DimProduct[ColorName],
            
            "Count",COUNTROWS(DimProduct)
            
    )

|ColorName      |	Count|
|---------------|--------|
|Azure          |	14|

Example 4: Filtering can happend by a different column within the same table as the FILTERS column

Colour Filters_filtered by product brand filter = 

    SUMMARIZE(
    
        CALCULATETABLE(
            
            FILTERS(DimProduct[ColorName]),
            
            FILTER(
                
                DimProduct,
                
                DimProduct[BrandName]="Adventure Works")),
            
            DimProduct[ColorName],
            
            "Count",COUNTROWS(DimProduct)
            
        )

|ColorName	|Count|
|-----------|-----|
|Silver	    |417|
|Black	    |602|
|Brown	    |77|
|White	    |505|
|Grey	    |283|
|Blue	    |200|
|Red	    |99|

Example 5: Filters table can be used to cross-filter by a column in a different table by using FILTER as long as the tables have a relationship

Colour Filters_cross_filtered with product filter = 

    SUMMARIZE(
    
        CALCULATETABLE(
            
            FILTERS(DimProduct[ColorName]),
            
            FILTER(
                
                DimGeography,
                
                DimGeography[RegionCountryName]="United States")
            
            ),
            
        DimProduct[ColorName],
        
        "Sales",FactOnlineSales[Sales_Amount]
    
)